In [3]:
import evaluate
import numpy as np
import nltk
import wandb

from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset
from huggingface_hub import notebook_login


# define constants
MODEL_ID = "google/flan-t5-base"
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
TRAIN_EPOCHS = 5
PAD_TOKEN = -100
MAX_INPUT_LENGTH = 512

DATA_LOCATION = "../data/"

In [ ]:
# load data
dataset = load_dataset('json', data_files={'train': [f'{DATA_LOCATION}train.json'],
                                           'val': [f'{DATA_LOCATION}val.json'],
                                         'test': [f'{DATA_LOCATION}test.json']})

In [5]:
# init tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id = PAD_TOKEN)

In [ ]:
# init peft config for faster training
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# tokenize the dataset
def apply_tokenization(row):
    input_statement = row['text']
    model_inputs = tokenizer(input_statement, max_length=MAX_INPUT_LENGTH, padding="max_length", truncation=True)
    labels = tokenizer(row["label"], max_length=8, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(apply_tokenization)
dataset = dataset.remove_columns(["text","label"])
dataset = dataset.with_format("torch")

In [8]:
# get preprocessed data splits
train_data = dataset['train']
val_data = dataset['val']

In [45]:
#OPTIONAL: select smaller subset of data for fast testing
train_subset = train_data.shuffle(seed=42).select(range(100))
val_subset = val_data.shuffle(seed=42).select(range(10))

In [ ]:
# test code to see if data was tokenized correctly
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)

batch = next(iter(train_dataloader))
print(batch.keys())
tokenizer.decode(batch['input_ids'][0])

In [11]:
# define evaluation metric
# taken from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/run_summarization.py
metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]
        
    # Replace -100s used for padding as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [14]:
# login to wandb for hyperparameter logging
wandb.login()

True

In [ ]:
# login to HF so we can push the finetuned model to HF
notebook_login()

In [48]:
# define training args and trainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5results",
    evaluation_strategy = "epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    save_total_limit=3,
    num_train_epochs=TRAIN_EPOCHS,
    predict_with_generate=True,
    # needs to be false to prevent overflow issues
    fp16=False,
    report_to="wandb",
    run_name=f"B-{BATCH_SIZE}_L-{LEARNING_RATE}_E-{TRAIN_EPOCHS}_M-{MAX_INPUT_LENGTH}"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# start training
trainer.train()

In [ ]:
# push the finetuned model and tokenizer to HF
model.push_to_hub("OvrK12/t5Seq2SeqBase1027")
tokenizer.push_to_hub("OvrK12/t5Seq2SeqBase1027")